# **FINAL PROJECT**

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Define three diverse models
def create_cnn_model():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_dnn_model():
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lenet_model():
    model = Sequential([
        Conv2D(20, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
        AveragePooling2D(pool_size=(2, 2)),
        Conv2D(50, (5, 5), activation='relu'),
        AveragePooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(500, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize and train the models
models = [create_cnn_model(), create_dnn_model(), create_lenet_model()]
model_performance = []

for model in models:
    model.fit(x_train, y_train_cat, epochs=10, verbose=0, validation_split=0.1)
    y_pred_prob = model.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    model_performance.append({
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

# Ensemble predictions by averaging
predictions = [model.predict(x_test) for model in models]
avg_predictions = np.mean(predictions, axis=0)
avg_labels = np.argmax(avg_predictions, axis=1)

ensemble_accuracy = accuracy_score(y_test, avg_labels)
ensemble_precision = precision_score(y_test, avg_labels, average='macro')
ensemble_recall = recall_score(y_test, avg_labels, average='macro')
ensemble_f1 = f1_score(y_test, avg_labels, average='macro')

# Print individual model performances
for i, performance in enumerate(model_performance, 1):
    print(f"Model {i} - Accuracy: {performance['accuracy']:.4f}, Precision: {performance['precision']:.4f}, "
          f"Recall: {performance['recall']:.4f}, F1-score: {performance['f1_score']:.4f}")

# Print ensemble performance
print(f"Ensemble - Accuracy: {ensemble_accuracy:.4f}, Precision: {ensemble_precision:.4f}, "
      f"Recall: {ensemble_recall:.4f}, F1-score: {ensemble_f1:.4f}")


313/313 [==============================] - 4s 11ms/step
Model 1 - Accuracy: 0.9127, Precision: 0.9126, Recall: 0.9127, F1-score: 0.9126
Model 2 - Accuracy: 0.8826, Precision: 0.8824, Recall: 0.8826, F1-score: 0.8815
Model 3 - Accuracy: 0.9067, Precision: 0.9056, Recall: 0.9067, F1-score: 0.9058
Ensemble - Accuracy: 0.9216, Precision: 0.9209, Recall: 0.9216, F1-score: 0.9209
